# Machine Learning in Chess to Study Patterns in Chess Games

## Import Libraries

In [22]:
import pandas as pd
import dask.dataframe as dd

## Load the Data

In [42]:
CSV_PATH = "/Users/isaac/Downloads/ChessDBs/lichess_db_standard_rated_2022.csv"
# Check whether the DataFrame has been loaded correctly.
df = dd.read_csv(CSV_PATH)
print(f"Loaded {len(df)} Rated Bullet, Rated Blitz, and Rated Rapid games from 2022.\n")
display(df.info())
display(df.head())

Loaded 63927037 Rated Bullet / Blitz / Rapid games from 2022.

<class 'dask.dataframe.core.DataFrame'>
Columns: 13 entries, UTCDate to Site
dtypes: object(11), int64(2)

None

,UTCDate,UTCTime,Event,TimeControl,Result,Termination,ECO,Opening,White,Black,WhiteElo,BlackElo,Site
0,2022.01.01,00:00:11,Rated Bullet game,120+1,0-1,Normal,D02,Queen's Pawn Game: Chigorin Variation,Kike73,Aox066,1255,1252,https://lichess.org/BPNsj2LF
1,2022.01.01,00:00:11,Rated Bullet game,120+1,1-0,Normal,C40,King's Pawn Game: Busch-Gass Gambit,schachueli,HiddenEyes,1346,1308,https://lichess.org/ooDjyGrm
2,2022.01.01,00:00:11,Rated Bullet game,120+1,0-1,Time forfeit,C02,"French Defense: Advance Variation, Nimzowitsch...",Renf7,Euphorbe92,859,964,https://lichess.org/T4TIdZZM
3,2022.01.01,00:00:11,Rated Bullet game,120+1,0-1,Time forfeit,B10,Caro-Kann Defense: Accelerated Panov Attack,Ralza63,FN2718,1769,1702,https://lichess.org/Oz5OFohi
4,2022.01.01,00:00:11,Rated Bullet game,120+1,1-0,Time forfeit,A41,Queen's Pawn Game,arkchess1963,sirkkeli,2146,2196,https://lichess.org/iehbAAzy


## Get Most Popular Openings

In [26]:
display(df["Opening"].value_counts().compute())

Modern Defense                                                       1643325
Queen's Pawn Game                                                    1636105
Scandinavian Defense: Mieses-Kotroc Variation                        1400483
Caro-Kann Defense                                                    1231717
Philidor Defense                                                     1154531
                                                                      ...   
Italian Game: Evans Gambit, Compromised Defense, Potter Variation          1
King's Pawn Game: Wayward Queen Attack, Mellon Gambit                      1
Ruy Lopez: Open, Bernstein Variation, Luther Line                          1
King's Gambit Accepted: Bishop's Gambit, McDonnell Attack                  1
King's Gambit Accepted: Bishop's Gambit, Chigorin's Attack                 1
Name: Opening, Length: 2998, dtype: int64

## Get Most Popular Opening Categories

In [27]:
display(df["ECO"].value_counts().compute())

A00    3951040
A40    3527282
B01    3384668
D00    2822607
B00    2794027
        ...   
D68         49
E96         24
D65         15
ECO         12
D99          1
Name: ECO, Length: 498, dtype: int64

## Get Elo Distribution of Players

In [17]:
# Get the Elo distribution of players.
